In [1]:
from pathlib import Path
from tqdm import tqdm
import napari
import tifffile
import numpy as np
from cellpose import models
import skimage
from skimage import measure
from scipy.ndimage import binary_erosion, binary_dilation, label
import pyclesperanto_prototype as cle
from apoc import ObjectSegmenter
import pandas as pd
from utils import count_particles_in_nuclei

In [2]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.lsm"):
    images.append(str(file_path))
    
images

['raw_data\\DSB Iba1 16_40X_CA1.lsm',
 'raw_data\\DSB Iba1 16_40X_CA3.lsm',
 'raw_data\\DSB Iba1 16_40X_CTX1.lsm',
 'raw_data\\DSB Iba1 16_40X_CTX2.lsm',
 'raw_data\\DSB Iba1 16_40X_CTX3.lsm',
 'raw_data\\DSB Iba1 16_40X_DG.lsm',
 'raw_data\\DSB Iba1 17_40X_CA1.lsm',
 'raw_data\\DSB Iba1 17_40X_CA3.lsm',
 'raw_data\\DSB Iba1 17_40X_CTX1.lsm',
 'raw_data\\DSB Iba1 17_40X_CTX2.lsm',
 'raw_data\\DSB Iba1 17_40X_CTX3.lsm',
 'raw_data\\DSB Iba1 17_40X_DG.lsm',
 'raw_data\\DSB Iba1 18_40X_CA1.lsm',
 'raw_data\\DSB Iba1 18_40X_CA3.lsm',
 'raw_data\\DSB Iba1 18_40X_CTX1.lsm',
 'raw_data\\DSB Iba1 18_40X_CTX2.lsm',
 'raw_data\\DSB Iba1 18_40X_CTX3.lsm',
 'raw_data\\DSB Iba1 18_40X_DG.lsm',
 'raw_data\\DSB Iba1 19_40X_CA1.lsm',
 'raw_data\\DSB Iba1 19_40X_CA3.lsm',
 'raw_data\\DSB Iba1 19_40X_CTX1.lsm',
 'raw_data\\DSB Iba1 19_40X_CTX2.lsm',
 'raw_data\\DSB Iba1 19_40X_CTX3.lsm',
 'raw_data\\DSB Iba1 19_40X_DG.lsm',
 'raw_data\\DSB Iba1 20_40X_CA1.lsm',
 'raw_data\\DSB Iba1 20_40X_CA3.lsm',
 'ra

In [10]:
viewer = napari.Viewer(ndisplay=2)
model = models.Cellpose(gpu=True, model_type='nuclei')

sigmas = [5, 6, 7, 8 , 9, 10]

for sigma in sigmas:
    
    image = images[47]
    
    # Extract filename
    file_path = Path(image)
    filename = file_path.stem

    # Read the image file 
    img = tifffile.imread(image)

    # Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
    nuclei_stack = img[:,0,:,:]
    dna_damage_stack = img[:,1,:,:]
    microglia_stack = img[:,2,:,:]

    # Perform maximum intensity projections
    nuclei_mip = np.max(nuclei_stack, axis=0)
    dna_damage_mip = np.max(dna_damage_stack, axis=0)
    microglia_mip = np.max(microglia_stack, axis=0)

    # Stack the arrays along a new axis (axis 0)
    stack = np.stack((nuclei_mip, dna_damage_mip, microglia_mip), axis=0)

    # Create a copy of nuclei_mip
    input_img = nuclei_mip.copy()

    # Might need to perform a Gaussian-blur before
    post_gaussian_img = skimage.filters.gaussian(input_img, sigma=sigma)

    # Predict nuclei nuclei_masks using cellpose
    nuclei_masks, flows, styles, diams = model.eval(post_gaussian_img, diameter=30, channels=[0, 0], net_avg=False)

    # Dilate nuclei to make sure the dna_damage_foci objects sit inside
    dilated_nuclei_masks = cle.dilate_labels(nuclei_masks, radius=4)
    # Erode dilated_nuclei_masks to obtain separate objects upon binarization
    eroded_nuclei_masks = cle.erode_labels(dilated_nuclei_masks, radius=1)

    # Show the resulting stack in Napari
    viewer.add_image(nuclei_mip, name=f"{filename}_nuclei")
    viewer.add_labels(eroded_nuclei_masks, name=f"{filename}_nuclei_mask_split_{sigma}")